In [3]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('data/2yrs_data.csv')
df

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Hour,is_weekend,is_holiday
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,17,1,0
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,17,1,0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,17,1,0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,17,1,0
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,17,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
543656,2007-12-31,23:55:00,1.746,0.204,242.41,7.2,0.0,0.0,18.0,23,0,0
543657,2007-12-31,23:56:00,1.732,0.210,242.42,7.2,0.0,0.0,18.0,23,0,0
543658,2007-12-31,23:57:00,1.732,0.210,242.50,7.2,0.0,0.0,18.0,23,0,0
543659,2007-12-31,23:58:00,1.684,0.144,242.18,7.0,0.0,0.0,18.0,23,0,0


# Baseline

In [15]:
df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour  
df['Minute'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.minute 
df['Day'] = pd.to_datetime(df['Date']).dt.day                      
df['Month'] = pd.to_datetime(df['Date']).dt.month                 
df['Year'] = pd.to_datetime(df['Date']).dt.year  

X = df[['Hour', 'Minute', 'Day', 'Month', 'Year', 'is_weekend','is_holiday']]
y = df['Global_active_power']    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
train_dmatrix = xgb.DMatrix(X_train, label=y_train)
test_dmatrix = xgb.DMatrix(X_test, label=y_test)

In [23]:
params = {
    'objective': 'reg:squarederror',  # Regression problem
    'max_depth': 20,                  # Tree depth
    'eta': 0.1,                      # Learning rate
    'seed': 42
}

model = xgb.train(params, train_dmatrix, num_boost_round=100)
y_pred = model.predict(test_dmatrix)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 0.32
